# ML Enabled Mobile CGE


## 1. Building Damage Analysis

The following code is preparing the IN-CORE analysis by checking versions and connecting to IN-CORE web service.


In [ ]:
from pyincore import IncoreClient, Dataset, FragilityService, MappingSet, DataService
from pyincore.analyses.buildingdamage import BuildingDamage
from pyincore.analyses.cumulativebuildingdamage import CumulativeBuildingDamage
from pyincore.analyses.montecarlofailureprobability import MonteCarloFailureProbability

import os
import pandas as pd
import numpy as np
import geopandas as gpd  # For reading in shapefiles
import matplotlib.pyplot as plt
from IPython.display import display

import sys  # For displaying package versions
import os  # For managing directories and file paths if drive is mounted

from pyincore_viz.geoutil import GeoUtil as viz
from pyincore_viz.plotutil import PlotUtil as plot

client = IncoreClient()
client.clear_cache()

In [ ]:
# create data_service object for loading files
data_service = DataService(client)

In [ ]:
# Check package versions - good practice for replication
print("Python Version ", sys.version)
print("pandas version: ", pd.__version__)
print("numpy version: ", np.__version__)

In [ ]:
# Mobile Building Inventory from IN-CORE
Building_Inventory_id = "6827560d1d2b050c74df1526"

# visualize the building inventory
Mobile_Building_Inventory = Dataset.from_data_service(
    Building_Inventory_id, DataService(client)
)
viz.plot_map(
    Mobile_Building_Inventory, column="archetype", category=True, basemap=True
)

In [ ]:
### This CELL will be removed in the future
building_to_sectors_id = "684877df9af67e766267ead7"
building_to_sectors_df = Dataset.from_data_service(
    building_to_sectors_id, DataService(client)
).get_dataframe_from_csv()

mobile_df = Mobile_Building_Inventory.get_dataframe_from_shapefile()
# check the columns in the building inventory
print("Columns in the building inventory: ", mobile_df.columns)
# check the columns in the building to sectors
print("Columns in the building to sectors: ", building_to_sectors_df.columns)
# join based on guid
matching = mobile_df.merge(building_to_sectors_df, on="guid", how="left")

print("number of buildings in the inventory: ", len(mobile_df))
print("Number of buildings in the inventory: ", len(matching))

In [ ]:
# load building inventory as Geodataframe
filename = Mobile_Building_Inventory.get_file_path("shp")
print(
    "The IN-CORE Dataservice has saved the Building Inventory on your local machine: "
    + filename
)
bldg_inv_gdf = gpd.read_file(filename)
bldg_inv_gdf.head()

In [ ]:
# Galveston Hurricane flood fragility mappings for buildings
mapping_id ="62fefd688a30d30dac57bbd7"
fragility_service = FragilityService(client)
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))

In [ ]:
# Reading the Hazard type and the Hazard ID for the six hurricane and SLR scenarios
hazard_type = "hurricane" 
hazard_id = "6420c6e6d9ae37665ff0d5f2" #Natural Katrina

### 1.2 Building Damage Analysis


In [ ]:
bldg_dmg = BuildingDamage(client)

bldg_dmg.load_remote_input_dataset("buildings", Building_Inventory_id)
bldg_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)

In [ ]:
# Run the building damage analysis
result_name = "Mobile_bldg_hurricane_dmg_result"

bldg_dmg.set_parameter("fragility_key", "Non-Retrofit Fragility ID Code")
bldg_dmg.set_parameter("result_name", result_name)
bldg_dmg.set_parameter("hazard_type", hazard_type)
bldg_dmg.set_parameter("hazard_id", hazard_id)
bldg_dmg.set_parameter("num_cpu", 4)
bldg_dmg.run_analysis()

In [ ]:
building_dmg_result = bldg_dmg.get_output_dataset("ds_result")

In [ ]:
# Convert dataset to Pandas DataFrame
bdmg_df = building_dmg_result.get_dataframe_from_csv(low_memory=False)

# Display top 5 rows of output data
bdmg_df.head()

## 2. Montecarlo Limit State Probability


In [ ]:
from pyincore.analyses.montecarlofailureprobability import MonteCarloFailureProbability

In [ ]:
num_samples = (
    500  # Require 500 samples for convergence - Selected smaller samples for testing
)
result_name = "Mobile_mc_failure_probability_buildings_dmg_result"

mc_bldg = MonteCarloFailureProbability(client)

mc_bldg.set_input_dataset("damage", building_dmg_result)
mc_bldg.set_parameter("num_cpu", 8)
mc_bldg.set_parameter("num_samples", num_samples)
mc_bldg.set_parameter("damage_interval_keys", ["DS_0", "DS_1", "DS_2", "DS_3"])
mc_bldg.set_parameter("failure_state_keys", ["DS_2", "DS_3"])

mc_bldg.set_parameter("result_name", result_name)

In [ ]:
mc_bldg.run_analysis()

# Obtain buildings failure probabilities
building_failure_probability = mc_bldg.get_output_dataset("failure_probability")

df_bldg_fail = building_failure_probability.get_dataframe_from_csv()
df_bldg_fail.head()

In [ ]:
# Obtain buildings sample damage states
sample_damage_states = mc_bldg.get_output_dataset("sample_damage_states")
df_bldg_ds = sample_damage_states.get_dataframe_from_csv()
df_bldg_ds.head()

In [ ]:
building_damage_mcs_samples = mc_bldg.get_output_dataset(
    "sample_failure_state"
)  # get buildings failure states

bdmcs = building_damage_mcs_samples.get_dataframe_from_csv()
bdmcs.head()

# 3. Capital Stock Shock and CGE Analysis


In [ ]:
from pyincore.analyses.capitalshocks import CapitalShocks

In [ ]:
# Joplin building to sector mapping table
building_to_sectors_id = "684877df9af67e766267ead7"
# Create Capital shocks analysis
capital_shocks = CapitalShocks(client)

# Load remote datasets
capital_shocks.load_remote_input_dataset("buildings_to_sectors", building_to_sectors_id)
# Set datasets
# Joplin building inventory
capital_shocks.load_remote_input_dataset("buildings", Building_Inventory_id)
# Joplin building failure probability
capital_shocks.set_input_dataset("failure_probability", building_failure_probability)

capital_shocks.set_parameter(
    "result_name", "sector_shocks"
)  # name of csv file with results

In [ ]:
# Run capital shocks analysis
capital_shocks.run_analysis()

sector_shocks_result = capital_shocks.get_output_dataset("sector_shocks")
sector_shocks_result.get_dataframe_from_csv()

### 4. ML Enabled CGE - Mobile


In [ ]:
from pyincore.analyses.mlenabledcgemobile import MlEnabledCgeMobile

In [ ]:
mlcgemobile = MlEnabledCgeMobile(client)
mlcgemobile.set_input_dataset("sector_shocks", sector_shocks_result)
mlcgemobile.set_parameter("result_name", "test_mobile_mlcge_result")

In [ ]:
mlcgemobile.run_analysis()

In [ ]:
domestic_supply = mlcgemobile.get_output_dataset("domestic-supply")
ds = domestic_supply.get_dataframe_from_csv()
# Display top 6 rows of output data
ds.head(6)



In [ ]:
gross_income = mlcgemobile.get_output_dataset("gross-income")
gi = gross_income.get_dataframe_from_csv()
gi.head()

In [ ]:
pre_factor_demand = mlcgemobile.get_output_dataset("pre-disaster-factor-demand")
pre_fd = pre_factor_demand.get_dataframe_from_csv()
pre_fd.head()

In [ ]:
post_factor_demand = mlcgemobile.get_output_dataset("post-disaster-factor-demand")
pos_fd = post_factor_demand.get_dataframe_from_csv()
pos_fd.head()

In [ ]:
household_count = mlcgemobile.get_output_dataset("household-count")
hc = household_count.get_dataframe_from_csv()
hc.head()